# More TA-Lib Indicators
#### Good Indicators
###### LINEARREG_INTERCEPT +
###### MEDPRICE +
###### TRANGE +
###### TYPPRICE +
###### MINUS_DM +
###### PLUS_DM +
#### Borderline Indicators
###### PPO -

In [ ]:
#quantopian imports
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar, Fundamentals
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume,SimpleBeta, Returns, RSI,EWMA
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises
from quantopian.pipeline.data import factset
from quantopian.pipeline.data.psychsignal import stocktwits

#Python imports
import math
import talib
import numpy as np
import pandas as pd
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics
from scipy.stats.mstats import winsorize
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)
from scipy.stats.mstats import gmean
from sklearn.cluster import SpectralClustering
 
from collections import Counter
import talib

In [ ]:
"""
HELPER FUNCTIONS
"""

def plus_dm_helper(high, low):
    """
    Returns positive directional movement. Abstracted for use with more complex factors

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices

    Returns
    -------
    np.array : matrix of positive directional movement

    """
    # get daily differences between high prices
    high_diff = (high - np.roll(high, 1, axis=0))[1:]

    # get daily differences between low prices
    low_diff = (np.roll(low, 1, axis=0) - low)[1:]

    # matrix of positive directional movement
    return np.where(((high_diff > 0) | (low_diff > 0)) & (high_diff > low_diff), high_diff, 0.)

def minus_dm_helper(high, low):
    """
    Returns negative directional movement. Abstracted for use with more complex factors

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices

    Returns
    -------
    np.array : matrix of negative directional movement

    """
    # get daily differences between high prices
    high_diff = (high - np.roll(high, 1, axis=0))[1:]

    # get daily differences between low prices
    low_diff = (np.roll(low, 1, axis=0) - low)[1:]

    # matrix of megative directional movement
    return np.where(((high_diff > 0) | (low_diff > 0)) & (high_diff < low_diff), low_diff, 0.)


def trange_helper(high, low, close):
    """
    Returns true range

    http://www.macroption.com/true-range/

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices
    close: np.array
        matrix of close prices

    Returns
    -------
    np.array : matrix of true range

    """
    # define matrices to be compared
    close = close[:-1]
    high = high[1:]
    low = low[1:]

    # matrices for comparison
    high_less_close = high - close
    close_less_low = close - low
    high_less_low = high - low

    # return maximum value for each cel
    return np.maximum(high_less_close, close_less_low, high_less_low)



In [ ]:
class BOP(CustomFactor):
    """
    Balance of Power

    Momentum indicator

    **Default Inputs:** USEquityPricing.close, USEquityPricing.open, USEquityPricing.high, USEquityPricing.low

    **Default Window Length:** 1

    https://www.interactivebrokers.com/en/software/tws/usersguidebook/technicalanalytics/balancepower.htm
    """
    inputs = [USEquityPricing.close, USEquityPricing.open, USEquityPricing.high, USEquityPricing.low]
    window_length = 1

    def compute(self, today, assets, out, close, open, high, low):
        out[:] = (close - open) / (high - low)
        

class LINEARREG_SLOPE(CustomFactor):
    """
    Slope of Trendline

    Momentum indicator.

    **Default Inputs:**  USEquitypricing.close

    **Default Window Length:** 14

    http://www.stat.cmu.edu/~cshalizi/mreg/15/lectures/06/lecture-06.pdf
    """	    
    inputs=[USEquityPricing.close]
    window_length=21

    # using MLE for speed
    def compute(self, today, assets, out, close):

        # prepare X matrix (x_is - x_bar)
        X = range(self.window_length)
        X_bar = np.nanmean(X)
        X_vector = X - X_bar
        X_matrix = np.tile(X_vector, (len(close.T), 1)).T

        # prepare Y matrix (y_is - y_bar)
        Y_bar = np.nanmean(close, axis=0)
        Y_bars = np.tile(Y_bar, (self.window_length, 1))
        Y_matrix = close - Y_bars

        # prepare variance of X
        X_var = np.nanvar(X)

        # multiply X matrix an Y matrix and sum (dot product)
        # then divide by variance of X
        # this gives the MLE of Beta
        out[:] = (np.sum((X_matrix * Y_matrix), axis=0) / X_var) / (self.window_length)


class LINEARREG_INTERCEPT(CustomFactor):
    """
    Intercept of Trendline

    **Default Inputs:**  USEquitypricing.close

    **Default Window Length:** 14

    http://www.stat.cmu.edu/~cshalizi/mreg/15/lectures/06/lecture-06.pdf
    """
    inputs=[USEquityPricing.close]
    window_length=21

    # using MLE
    def compute(self, today, assets, out, close):

        # prepare X matrix (x_is - x_bar)
        X = range(self.window_length)
        X_bar = np.nanmean(X)
        X_vector = X - X_bar
        X_matrix = np.tile(X_vector, (len(close.T), 1)).T

        # prepare Y vectors (y_is - y_bar)
        Y_bar = np.nanmean(close, axis=0)
        Y_bars = np.tile(Y_bar, (self.window_length, 1))
        Y_matrix = close - Y_bars
        
        # prepare variance of X
        X_var = np.nanvar(X)

        # multiply X matrix an Y matrix and sum (dot product)
        # then divide by variance of X
        # this gives the MLE of Beta
        betas = (np.sum((X_matrix * Y_matrix), axis=0) / X_var) / (self.window_length)

        # now use to get to MLE of alpha
        out[:] = Y_bar - (betas * X_bar)

class MEDPRICE(CustomFactor):
    """
    Mean of a day's high and low prices

    **Default Inputs:**  USEquityPricing.high, USEquityPricing.low

    **Default Window Length:** 1

    http://www.fmlabs.com/reference/default.htm?url=MedianPrices.htm
    """	    
    inputs = [USEquityPricing.high, USEquityPricing.low]
    window_length = 1

    def compute(self, today, assets, out, high, low):
        out[:] = (high + low) / 2.
        

class TRANGE(CustomFactor):
    """
    True Range 

    **Default Inputs:**  USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

    **Default Window Length:** 2

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/atr
    """    
    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 2

    def compute(self, today, assets, out, high, low, close):
        out[:] = np.nanmax([(high[-1] - close[0]), (close[0] - low[-1]), (high[-1] - low[-1])], axis=0)

class MACD_Signal_10d(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 60

        def compute(self, today, assets, out, close):

            sig_lines = []

            for col in close.T:
                # get signal line only
                try:
                    _, signal_line, _ = talib.MACD(col, fastperiod=12,
                                                   slowperiod=26, signalperiod=10)
                    sig_lines.append(signal_line[-1])
                # if error calculating, return NaN
                except:
                    sig_lines.append(np.nan)
            out[:] = sig_lines
            

# 20-day Stochastic Oscillator
class Stochastic_Oscillator(CustomFactor):
    """
    20-day Stochastic Oscillator:
    K = (close price - 5-day low) / (5-day high - 5-day low)
    D = 100 * (average of past 3 K's)
    We use the slow-D period here (the D above)
    https://www.math.nyu.edu/faculty/avellane/Lo13030.pdf
    Notes:
    High value suggests turning point in positive momentum (expected decrease)
    Low value suggests turning point in negative momentum (expected increase)
    """
    inputs = [USEquityPricing.close,
              USEquityPricing.high, USEquityPricing.low]
    window_length = 30

    def compute(self, today, assets, out, close, high, low):

        stoch_list = []

        for col_c, col_h, col_l in zip(close.T, high.T, low.T):
            try:
                _, slowd = talib.STOCH(col_h, col_l, col_c,
                                       fastk_period=5, slowk_period=3, slowk_matype=0,
                                       slowd_period=3, slowd_matype=0)
                stoch_list.append(slowd[-1])
            # if error calculating
            except:
                stoch_list.append(np.nan)

        out[:] = stoch_list

In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
    
    # ALPHA FACTOR 1
    alpha_factor1 = BOP()
    
    # Standardized logic for each input factor after this point
    alpha_w1 = alpha_factor1.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor1.isfinite())
    
    alpha_z1 = alpha_w1.zscore()
    alpha_weight1 = alpha_z1 / 100.0
    
    outlier_filter1 = alpha_z1.abs() < ZSCORE_FILTER
    zero_filter1 = alpha_weight1.abs() > ZERO_FILTER
 
    universe1 = QTradableStocksUS() & \
               outlier_filter1 & \
               zero_filter1
    
    # ALPHA FACTOR 2
    alpha_factor2 = LINEARREG_SLOPE()
    
    # Standardized logic for each input factor after this point
    alpha_w2 = alpha_factor2.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor2.isfinite())
    
    alpha_z2 = alpha_w2.zscore()
    alpha_weight2 = alpha_z2 / 100.0
    
    outlier_filter2 = alpha_z2.abs() < ZSCORE_FILTER
    zero_filter2 = alpha_weight2.abs() > ZERO_FILTER
 
    universe2 = QTradableStocksUS() & \
               outlier_filter2 & \
               zero_filter2
    
    # ALPHA FACTOR 3
    alpha_factor3 = LINEARREG_INTERCEPT()
    
    # Standardized logic for each input factor after this point
    alpha_w3 = alpha_factor3.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor3.isfinite())
    
    alpha_z3 = alpha_w3.zscore()
    alpha_weight3 = alpha_z3 / 100.0
    
    outlier_filter3 = alpha_z3.abs() < ZSCORE_FILTER
    zero_filter3 = alpha_weight3.abs() > ZERO_FILTER
 
    universe3 = QTradableStocksUS() & \
               outlier_filter3 & \
               zero_filter3
    
    # ALPHA FACTOR 4
    alpha_factor4 = MEDPRICE()
    
    # Standardized logic for each input factor after this point
    alpha_w4 = alpha_factor4.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor4.isfinite())
    
    alpha_z4 = alpha_w4.zscore()
    alpha_weight4 = alpha_z4 / 100.0
    
    outlier_filter4 = alpha_z4.abs() < ZSCORE_FILTER
    zero_filter4 = alpha_weight4.abs() > ZERO_FILTER
 
    universe4 = QTradableStocksUS() & \
               outlier_filter4 & \
               zero_filter4
    
    # ALPHA FACTOR 5
    alpha_factor5 = MACD_Signal_10d()
    
    # Standardized logic for each input factor after this point
    alpha_w5 = alpha_factor5.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor5.isfinite())
    
    alpha_z5 = alpha_w5.zscore()
    alpha_weight5 = alpha_z5 / 100.0
    
    outlier_filter5 = alpha_z5.abs() < ZSCORE_FILTER
    zero_filter5 = alpha_weight5.abs() > ZERO_FILTER
 
    universe5 = QTradableStocksUS() & \
               outlier_filter5 & \
               zero_filter5
    
    
    # ALPHA FACTOR 6
    alpha_factor6 = TRANGE()
    
    # Standardized logic for each input factor after this point
    alpha_w6 = alpha_factor6.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor6.isfinite())
    
    alpha_z6 = alpha_w6.zscore()
    alpha_weight6 = alpha_z6 / 100.0
    
    outlier_filter6 = alpha_z6.abs() < ZSCORE_FILTER
    zero_filter6 = alpha_weight6.abs() > ZERO_FILTER
 
    universe6 = QTradableStocksUS() & \
               outlier_filter6 & \
               zero_filter6
    
    # ALPHA FACTOR 7
    alpha_factor7 =  Stochastic_Oscillator()
    
    # Standardized logic for each input factor after this point
    alpha_w7 = alpha_factor5.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor7.isfinite())
    
    alpha_z7 = alpha_w7.zscore()
    alpha_weight7 = alpha_z7 / 100.0
    
    outlier_filter7 = alpha_z7.abs() < ZSCORE_FILTER
    zero_filter7 = alpha_weight7.abs() > ZERO_FILTER
 
    universe7 = QTradableStocksUS() & \
               outlier_filter7 & \
               zero_filter7
    
    universe =  universe1 & universe2 & universe3 & universe4 & universe5 & universe6 & universe7
    
    alpha_weight = alpha_weight2 + alpha_weight3 + alpha_weight4 +alpha_weight5+ alpha_weight6 +alpha_weight7
    
    testing_quantiles = alpha_weight.quantiles(2)
    
    sector = Sector()
    
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'shorts':testing_quantiles.eq(0),
            'longs':testing_quantiles.eq(1),
            'BOP': alpha_weight1,
            'LINEARREG_SLOPE': alpha_weight2,
            'LINEARREG_INTERCEPT': alpha_weight3,
            'MEDPRICE': alpha_weight4,
            'TRANGE': alpha_weight5,
            'MACD_Signal_10d': alpha_weight6,
            'Stochastic_Oscillator': alpha_weight7,
            'sector': sector,
        },
        screen=universe
    )
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(), start_date = '2015-01-01', end_date = '2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1]
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2014-01-01', end_date = '2017-01-01',fields='open_price')

In [ ]:
import alphalens
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data1 = get_clean_factor_and_forward_returns(
    result['MACD_Signal_10d'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['Stochastic_Oscillator'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### BOP - weak

In [ ]:


factor_data1 = get_clean_factor_and_forward_returns(
    result['BOP'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)


#### LINEARREG_SLOPE - bad

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['LINEARREG_SLOPE'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### LINEARREG_INTERCEPT - GOOD

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['LINEARREG_INTERCEPT'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### MEDPRICE - GOOD

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['MEDPRICE'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### TRANGE - decent

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['TRANGE'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

# Directional Indicators

In [ ]:

class MINUS_DI(CustomFactor):
    """
    Negative directional indicator

    Momentum indicator

    **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

    **Default Window Length:** 15

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
    """	

    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 22

    def compute(self, today, assets, out, high, low, close):
            out[:] = 100 * np.sum(minus_dm_helper(high, low), axis=0) / np.sum(trange_helper(high, low, close), axis=0)


class MINUS_DM(CustomFactor):
    """
    Negative directional movement

    Momentum indicator

    **Default Inputs:** USEquityPricing.high, USEquityPricing.low

    **Default Window Length:** 15

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
    """	

    inputs = [USEquityPricing.high, USEquityPricing.low]
    window_length = 22

    def compute(self, today, assets, out, high, low):
            out[:] = np.sum(minus_dm_helper(high, low), axis=0)


class PLUS_DI(CustomFactor):
    """
    Positive directional indicator

    Momentum indicator

    **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

    **Default Window Length:** 15

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
    """	
    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 22

    def compute(self, today, assets, out, high, low, close):
            out[:] = 100 * np.sum(plus_dm_helper(high, low), axis=0) / np.sum(trange_helper(high, low, close), axis=0)


class PLUS_DM(CustomFactor):
    """
    Positive directional movement

    Momentum indicator

    **Default Inputs:** USEquityPricing.high, USEquityPricing.low

    **Default Window Length:** 15

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
    """	
    inputs = [USEquityPricing.high, USEquityPricing.low]
    window_length = 22

    def compute(self, today, assets, out, high, low):
            out[:] = np.sum(plus_dm_helper(high, low), axis=0)


def PPO(fast_period=12, slow_period=26):
    """
    Function to produce CustomFactor of Percentage Price Oscillator
    Called in same way as a normal class, but used in order to give variable
    fast- and slow- periods

    Parameters
    ----------
    fast_period : int > 0
        shorter period moving average 
    slow_period : int > 0 (> fast_period)
        longer period moving average

    Returns
    -------
    zipline.CustomFactor : Percentage Price Oscillator factor

    **Default Inputs:**  12, 26

    http://www.investopedia.com/terms/p/ppo.asp
    """  
    class PPO_(CustomFactor):

        inputs=[USEquityPricing.close]
        window_length = slow_period

        def compute(self, today, assets, out, close):
            slowMA = np.mean(close, axis=0)
            fastMA = np.mean(close[-fast_period:], axis=0)
            out[:] = ((fastMA - slowMA) / slowMA) * 100.

    return PPO_()

class TYPPRICE(CustomFactor):
    """
    Typical Price 

    **Default Inputs:**  USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

    **Default Window Length:** 1

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/typical-price
    """    
    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 1

    def compute(self, today, assets, out, high, low, close):
        out[:] = (high + low + close) / 3.

In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
    
    # ALPHA FACTOR 1
    alpha_factor1 = MINUS_DI()
    
    # Standardized logic for each input factor after this point
    alpha_w1 = alpha_factor1.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor1.isfinite())
    
    alpha_z1 = alpha_w1.zscore()
    alpha_weight1 = alpha_z1 / 100.0
    
    outlier_filter1 = alpha_z1.abs() < ZSCORE_FILTER
    zero_filter1 = alpha_weight1.abs() > ZERO_FILTER
 
    universe1 = QTradableStocksUS() & \
               outlier_filter1 & \
               zero_filter1
    
    # ALPHA FACTOR 2
    alpha_factor2 = MINUS_DM()
    
    # Standardized logic for each input factor after this point
    alpha_w2 = alpha_factor2.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor2.isfinite())
    
    alpha_z2 = alpha_w2.zscore()
    alpha_weight2 = alpha_z2 / 100.0
    
    outlier_filter2 = alpha_z2.abs() < ZSCORE_FILTER
    zero_filter2 = alpha_weight2.abs() > ZERO_FILTER
 
    universe2 = QTradableStocksUS() & \
               outlier_filter2 & \
               zero_filter2
    
    # ALPHA FACTOR 3
    alpha_factor3 = PLUS_DI()
    
    # Standardized logic for each input factor after this point
    alpha_w3 = alpha_factor3.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor3.isfinite())
    
    alpha_z3 = alpha_w3.zscore()
    alpha_weight3 = alpha_z3 / 100.0
    
    outlier_filter3 = alpha_z3.abs() < ZSCORE_FILTER
    zero_filter3 = alpha_weight3.abs() > ZERO_FILTER
 
    universe3 = QTradableStocksUS() & \
               outlier_filter3 & \
               zero_filter3
    
    # ALPHA FACTOR 4
    alpha_factor4 = PLUS_DM()
    
    # Standardized logic for each input factor after this point
    alpha_w4 = alpha_factor4.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor4.isfinite())
    
    alpha_z4 = alpha_w4.zscore()
    alpha_weight4 = alpha_z4 / 100.0
    
    outlier_filter4 = alpha_z4.abs() < ZSCORE_FILTER
    zero_filter4 = alpha_weight4.abs() > ZERO_FILTER
 
    universe4 = QTradableStocksUS() & \
               outlier_filter4 & \
               zero_filter4
    
    # ALPHA FACTOR 5
    alpha_factor5 = PPO()
    
    # Standardized logic for each input factor after this point
    alpha_w5 = alpha_factor5.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor5.isfinite())
    
    alpha_z5 = alpha_w5.zscore()
    alpha_weight5 = alpha_z5 / 100.0
    
    outlier_filter5 = alpha_z5.abs() < ZSCORE_FILTER
    zero_filter5 = alpha_weight5.abs() > ZERO_FILTER
 
    universe5 = QTradableStocksUS() & \
               outlier_filter5 & \
               zero_filter5
    
    # ALPHA FACTOR 6
    alpha_factor6 = TYPPRICE()
    
    # Standardized logic for each input factor after this point
    alpha_w6 = alpha_factor6.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor6.isfinite())
    
    alpha_z6 = alpha_w6.zscore()
    alpha_weight6 = alpha_z6 / 100.0
    
    outlier_filter6 = alpha_z6.abs() < ZSCORE_FILTER
    zero_filter6 = alpha_weight6.abs() > ZERO_FILTER
 
    universe6 = QTradableStocksUS() & \
               outlier_filter6 & \
               zero_filter6
    
    universe =  universe1 & universe2 & universe3 & universe4 & universe5 & universe6
    
    alpha_weight = alpha_weight2 + alpha_weight3 + alpha_weight4 +alpha_weight5+alpha_weight6
    
    testing_quantiles = alpha_weight.quantiles(2)
    
    sector = Sector()
    
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'shorts':testing_quantiles.eq(0),
            'longs':testing_quantiles.eq(1),
            'MINUS_DI': alpha_weight1,
            'MINUS_DM': alpha_weight2,
            'PLUS_DI': alpha_weight3,
            'PLUS_DM': alpha_weight4,
            'PPO': alpha_weight5,
            'TYPPRICE': alpha_weight6,
            'sector': sector,
        },
        screen=universe
    )
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(), start_date = '2015-01-01', end_date = '2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1]
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2014-01-01', end_date = '2017-01-01',fields='open_price')

#### MINUS_DI - borderline

In [ ]:
import alphalens
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data1 = get_clean_factor_and_forward_returns(
    result['MINUS_DI'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### MINUS_DM - good

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['MINUS_DM'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### PLUS_DI - good negative

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['PLUS_DI'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### PLUS_DM - good/borderline

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['PLUS_DM'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### PPO - borderline/bad

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['PPO'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### TYPPRICE - GOOD

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['TYPPRICE'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)